In [4]:
import torch
import matplotlib.pyplot as plt
from sklearn.neighbors import kneighbors_graph
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from sklearn.metrics.cluster import adjusted_rand_score as ARI
from sklearn.cluster import SpectralClustering
import cvxpy as cp
import warnings
warnings.filterwarnings('ignore')
from util import eval, loadDataset, normalization, kmeansInitialization

def QProg(S, gamma):
    K = len(S)
    
    # Define the vector s = [s_1, s_2, ..., s_K]
    s = cp.Parameter(K)
    
    # Create the optimization variable tau
    tau = cp.Variable(K)

    # Define the objective function
    objective = cp.sum(cp.multiply(tau, s)) + gamma * cp.norm(tau, 2)**2
    
    # Define the constraints
    constraints = [cp.sum(tau) == 1, tau >= 0]
    
    # Form the QP problem
    problem = cp.Problem(cp.Minimize(objective), constraints)
    
    # Solve the QP problem
    # First, assign values to the parameter s
    s.value = S
    problem.solve()

    return tau.value

eps = torch.tensor(10 ** -10)

datasets = ['3Sources.npy',
'BBCSport.npy',
'Caltech101.npy',
'Caltech_2.npy',
'Citeseer.npy',
'Coil100.npy',
'Cora.npy',
'EYaleB10.npy',
'Handwritten.npy',
'MNIST10.npy',
'UCIdigit.npy',
'Umist.npy',
'Yale32.npy',
'Yeast.npy',
'Cora2.npy',
'texas.npy',
'wisconsin.npy',
'washington.npy',
'cornell.npy']


X0, Y, V, c = loadDataset(10)


r = 50
k = 5
kg = 5
alpha = 1
beta = 0.001
iter = 300

tau = torch.ones(V)/V

W = []
H = []
X = []


for v in range(V):
    
    X.append(torch.tensor(X0[v]).type(torch.float32))
    
    d, n = X[v].shape


    W.append(torch.rand(d, r))
    H.append(torch.rand(r, n))

Hc = torch.rand(r, n)
    
A = []
D = []
for v in range(V):
    
    A0 = kneighbors_graph(X[v].T, k, mode='connectivity', include_self=False).toarray()
    A0 = torch.tensor(A0).type(torch.float32)
    A0 = torch.maximum(A0, A0.T)

    A.append(A0)
    D.append(torch.diag(torch.sum(A0, dim = 1)))

# Optimization
err = torch.zeros(iter)

for t in range(iter):
    
    # Updating Ws
    for v in range(V): 
        Wn = X[v]      @ Hc.T + X[v]        @ H[v].T
        Wd = W[v] @ Hc @ Hc.T + W[v] @ H[v] @ H[v].T
        W[v] = W[v] * (Wn / torch.maximum(Wd, eps)) ** 0.5
        
        # Normalization?
    
    # Updating H*
    Hn = torch.zeros(r, n)
    Hd = torch.zeros(r, n)
    Hsum = torch.zeros(r, n)
    for v in range(V):
        Hn += W[v].T @ X[v]      + alpha * (tau[v] * (Hc @ A[v]))
        Hd += W[v].T @ W[v] @ Hc + alpha * (tau[v] * (Hc @ D[v]))
        
    Hc = Hc * (Hn / torch.maximum(Hd, eps)) ** 0.5

    # Normalization?
    
    # Updating Hs
    for v in range(V):
        
        Hn = W[v].T @ X[v]        + alpha * (H[v] @ A[v])
        Hd = W[v].T @ W[v] @ H[v] + alpha * (H[v] @ D[v])
        H[v] = H[v] * (Hn / torch.maximum(Hd, eps)) ** 0.5
        
        # Normalization?

    # Updating Tau
    S = []
    for v in range(V):
        S.append(torch.trace(Hc @ (D[v] - A[v]) @ Hc.T))    
    tau = QProg(S, beta/alpha)

        # Calculating cost function
#         err[t] += torch.norm(X[v] - W[v] @ H[v]) ** 2 + torch.norm(X[v] - W[v] @ Hs) ** 2 \
#         + lam * torch.trace(H[v] @ Lopt @ H[v].T) + delta * torch.trace(Hs.T @ (Hsum/V))
           
# plt.plot(err)


Hfinal = []
for v in range(V):
    Hfinal.append(H[v])
Hfinal.append(Hc)

Hfinal = torch.concat(Hfinal, 0)


S = kneighbors_graph(Hfinal.T, kg, mode='connectivity', include_self=False).toarray()
S = torch.tensor(S).type(torch.float32)
S = torch.maximum(S, S.T)

# Spectral Clustering on the S matrix
pred = SpectralClustering(c, affinity='precomputed').fit(S).labels_

nmi, acc, ari, f1mi, f1ma, f1we, pur = eval(Y, pred)
nmi, acc, ari, f1mi, f1ma, f1we, pur

(0.898171765346944,
 0.951,
 0.8942266349931431,
 0.951,
 0.9511746801543877,
 0.9511746801543878,
 0.951)